<a href="https://colab.research.google.com/github/sujalarora19/DomainValuationNamekart/blob/main/UsingDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

In [2]:
df_data = pd.read_csv('Dataset.csv')
df=df_data.copy()
df.drop_duplicates(subset='domain',inplace=True)
df.rename(columns={'price':'currbid'},inplace=True)
df.drop(axis=1, columns=['priority_score','words'], inplace=True)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 785962 entries, 0 to 785961
Data columns (total 30 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   domain                            785962 non-null  object 
 1   category                          785962 non-null  object 
 2   suffix                            785962 non-null  object 
 3   whois_registrar                   785962 non-null  object 
 4   extensions_taken                  785962 non-null  int64  
 5   search_ad_competition_broad_term  785962 non-null  int64  
 6   search_ad_competition_exact_term  785962 non-null  int64  
 7   search_cpc_broad_term             785962 non-null  float64
 8   search_cpc_exact_term             785962 non-null  float64
 9   search_results_sld                785962 non-null  int64  
 10  search_results_term               785962 non-null  int64  
 11  search_volume_broad_term          785962 non-null  i

All are non-null \
No imputer needed

In [4]:
index_cols = ['domain',"currbid"]
df["log_currbid"] = np.log1p(df["currbid"])  # log(1 + x) avoids log(0)
target_col = 'log_currbid'

In [5]:

categorical_cols = [col for col in df.select_dtypes(include=['object']).columns if col not in index_cols]
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
numerical_cols.remove(target_col)
for i in index_cols:
  if i in numerical_cols:
    numerical_cols.remove(i)

# df[categorical_cols] = df[categorical_cols].fillna('<unk>')

In [6]:
X = df[numerical_cols + categorical_cols]

y = df[target_col]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)



encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X_train[categorical_cols] = encoder.fit_transform(X_train[categorical_cols])
X_test[categorical_cols] = encoder.transform(X_test[categorical_cols])


# num_imputer = SimpleImputer(strategy='mean')
# X_train[numerical_cols] = num_imputer.fit_transform(X_train[numerical_cols])
# X_test[numerical_cols] = num_imputer.transform(X_test[numerical_cols])


scaler = StandardScaler()
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

In [7]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Custom Dataset
class TabularDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X.values, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.float32).unsqueeze(1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = TabularDataset(X_train, y_train)
test_dataset = TabularDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=512)


In [8]:
class MLPRegressor(nn.Module):
    def __init__(self, input_dim):
        super(MLPRegressor, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.model(x)

model = MLPRegressor(X_train.shape[1]).to(device)


In [9]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

def train_model(model, loader):
    model.train()
    total_loss = 0
    for X_batch, y_batch in loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(loader)

def evaluate_model(model, loader):
    model.eval()
    predictions, targets = [], []
    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch = X_batch.to(device)
            outputs = model(X_batch)
            predictions.append(outputs.cpu())
            targets.append(y_batch)

    preds = torch.cat(predictions).squeeze().numpy()
    trues = torch.cat(targets).squeeze().numpy()
    return preds, trues


In [13]:
for epoch in range(9):
    train_loss = train_model(model, train_loader)
    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}")

log_preds, log_trues = evaluate_model(model, test_loader)

# Convert from log scale
preds_actual = np.expm1(log_preds)
true_actual = np.expm1(log_trues)


Epoch 1, Train Loss: 0.3020
Epoch 2, Train Loss: 0.3540
Epoch 3, Train Loss: 0.2786
Epoch 4, Train Loss: 0.2754
Epoch 5, Train Loss: 0.2813
Epoch 6, Train Loss: 0.2863
Epoch 7, Train Loss: 0.2797
Epoch 8, Train Loss: 0.2650
Epoch 9, Train Loss: 0.2661


In [15]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
real_rmse = np.sqrt(mean_squared_error(true_actual, preds_actual))
real_mae = mean_absolute_error(true_actual, preds_actual)

print(f"Real RMSE: {real_rmse:.2f}")
print(f"Real MAE: {real_mae:.2f}")

Real RMSE: 3158.69
Real MAE: 70.72
